# Equation system solution space

This script will look at our system of equations and change the parameters to try to visualize how changes to each of them affect the solution space, that is, the proportion of complexes that are HET.


In [1]:
# Import libraries
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
import seaborn as sns
import os
from datetime import date
from datetime import datetime
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# Define helper functions
#### Prepare the functions that will apply the effects of the sampled mutations
def calculate_folded_fraction2(curr_stab):
    '''This function receives a starting deltaG of subunit stability. 
    With this, it calculates the new deltaG of stability and the fraction of
    folded proteins.
    '''
    folded_fraction = 1 / (1 + math.exp(curr_stab))

    return(folded_fraction)

def calculate_eq_constant2(curr_binding_energy, R, T):
    '''This function receives a starting binding energy.
    With this, it calculates the new equilibrium constant of association.
    '''    
    # Calculate the new equilibrium constant
    K = math.exp(-1 * curr_binding_energy/ (R * T))
    
    return(K)

# Define functions
def mean(lst):
    return(round(sum(lst) / len(lst), 2))

In [3]:
def dist_to_pos(number):
    if number.real >= 0:
        return abs(number.imag)
    else:
        return abs(number)

In [4]:
## Define some variables and constants
# Constants
R = 1.987e-3 # kcal / (mol * K)
T = 298 # K
dAA = 1.3
dBB = 1.3
dAB = 1.3
dA = 1.3
dB = 1.3

# Variables
sA = 100
sB = 100

curr_binding_energy_AA = -10
curr_binding_energy_BB = -10
curr_binding_energy_AB = -10

## Parameters for the fitness curve
alpha = 60
beta = 0.5

# Parameters for stability values
curr_stab_A=-5
curr_stab_B=-5


In [5]:
# Define the main function we will test
def equation_system(sA, sB, curr_stab_A, curr_stab_B, curr_binding_energy_AA, curr_binding_energy_BB, curr_binding_energy_AB):
    # Calculate the folded fractions
    folded_fraction_A = calculate_folded_fraction2(curr_stab_A)
    folded_fraction_B = calculate_folded_fraction2(curr_stab_B)
        
    # Calculate association constant
    kAA = calculate_eq_constant2(curr_binding_energy_AA, R, T)
    kBB = calculate_eq_constant2(curr_binding_energy_BB, R, T)
    kAB = calculate_eq_constant2(curr_binding_energy_AB, R, T)
    
    # Use the folded fraction to update synthesis rates
    sA = sA*(folded_fraction_A)
    sB = sB*(folded_fraction_B)

    # Double kAB to account for the increased collision probability
    kAB = kAB*2

    # Coefficients
    coeff4 = 2*dAA*kAA*(4*dAA*kAA*dBB*kBB/(dAB*kAB) - dAB*kAB)

    coeff3 = dA*(8*dAA*kAA*dBB*kBB/(dAB*kAB) - dAB*kAB) - 2*dB*dAA*kAA

    coeff2 = 2*dBB*kBB*(dA**2)/(dAB*kAB) + sA*(dAB*kAB - 8*dAA*kAA*dBB*kBB/(dAB*kAB)) - dA*dB - dAB*kAB*sB

    coeff1 = sA*(dB - 4*dA*dBB*kBB/(dAB*kAB))

    coeff0 = 2*(sA**2)*dBB*kBB/(dAB*kAB)

    coeffs = [coeff4, coeff3, coeff2, coeff1, coeff0]
    
    # Look for the roots
    cA = np.roots(coeffs)

    # There is thus a vector of 5 possible values for each concentration.

    cB = (sA/cA - 2*dAA*kAA*cA - dA)/(dAB*kAB)

    cAA = (cA**2)*kAA

    cBB = (cB**2)*kBB

    cAB = cA*cB*kAB
    
    # State which as the smallest sum of distances of concentrations
    # from the positive real numbers.
    most_positive_state = min(
        zip(cA, cAA, cAB, cBB, cB),
        key=lambda state: sum(map(dist_to_pos, state))
    )

    if not np.isclose(sum(map(dist_to_pos, most_positive_state)), 0):
        print("The numerical solving of the equilibrium state did not find positive concentrations.")
        return(0, 0, 0, 0, 0)

    # Take the absolute value of concentrations
    cA, cAA, cAB, cBB, cB = tuple(map(abs, most_positive_state))
    
    # Return the concentrations we need
    return(cA, cB, cAA, cBB, cAB)

In [6]:
equation_system(sA, sB, curr_stab_A, curr_stab_B, curr_binding_energy_AA, curr_binding_energy_BB, curr_binding_energy_AB)

0.9933071490757153 0.9933071490757153
0.9933071490757153 0.9933071490757153


(0.0009403515538666324,
 0.0009403515538666326,
 19.1018254712599,
 19.10182547125991,
 38.20365094251981)

## Test the effects of mutations on stability

In [58]:
stab_A_values = np.arange(-10, 10, 0.01)
stab_B_values = np.arange(-10, 10, 0.01)


In [59]:
list_results = []
for new_stab_A in stab_A_values:

    for new_stab_B in stab_B_values:

        # Feed the values into the system of equations
        cA, cB, cAA, cBB, cAB = equation_system(sA, sB, new_stab_A, new_stab_B, curr_binding_energy_AA, curr_binding_energy_BB, curr_binding_energy_AB)

        # Create a new line
        if (cA + cB + cAA + cAB + cBB) == 0:
            pct_AB = 0
        else:
            pct_AB = 100*cAB / (cA + cB + cAA + cAB + cBB)
            
        new_line = [new_stab_A, new_stab_B, cA, cB, cAA, cBB, cAB, pct_AB]
        
        list_results.append(new_line)

    
# Convert the list of lists to a dataframe
df = pd.DataFrame(list_results, columns = ['new_stab_A', 'new_stab_B', 'cA', 'cB', 'cAA', 'cBB', 'cAB', 'pct_AB'])

In [60]:
df.to_csv('../../Data/Results_solution_space/solutionSpace_subunitStab.tsv', 
         sep = '\t', index = False)

,new_stab_A,new_stab_B,cA,cB,cAA,cBB,cAB,pct_AB
0,-10.0,-10.0,0.000944,0.000944,19.230533,19.230533,38.461067,49.998773
1,-10.0,-9.9,0.000944,0.000944,19.230533,19.230533,38.461067,49.998773
2,-10.0,-9.8,0.000944,0.000944,19.230533,19.230533,38.461067,49.998773
3,-10.0,-9.7,0.000944,0.000944,19.230533,19.230533,38.461067,49.998773
4,-10.0,-9.6,0.000944,0.000944,19.230533,19.230533,38.461067,49.998773
...,...,...,...,...,...,...,...,...
39995,9.9,9.5,0.000944,0.000944,19.230430,19.230489,38.460919,49.998773
39996,9.9,9.6,0.000944,0.000944,19.230433,19.230479,38.460912,49.998773
39997,9.9,9.7,0.000944,0.000944,19.230437,19.230469,38.460905,49.998773
39998,9.9,9.8,0.000944,0.000944,19.230440,19.230457,38.460898,49.998773


## Try with changes to the binding affinity of BB and AB

In [14]:
binding_AB_values = np.arange(-15 -5, 0.01)
binding_BB_values = np.arange(-15, -5, 0.01)


In [15]:
list_results = []
for new_binding_AB in binding_AB_values:

    for new_binding_BB in binding_BB_values:

        # Feed the values into the system of equations
        cA, cB, cAA, cBB, cAB = equation_system(sA, sB, curr_stab_A, curr_stab_B, curr_binding_energy_AA, new_binding_BB, new_binding_AB)

        # Create a new line
        if (cA + cB + cAA + cAB + cBB) == 0:
            pct_AB = 0
        else:
            pct_AB = 100*cAB / (cA + cB + cAA + cAB + cBB)
            
        new_line = [new_binding_AB, new_binding_BB, cA, cB, cAA, cBB, cAB, pct_AB]
        
        list_results.append(new_line)

    
# Convert the list of lists to a dataframe
df = pd.DataFrame(list_results, columns = ['new_binding_AB', 'new_binding_BB', 'cA', 'cB', 'cAA', 'cBB', 'cAB', 'pct_AB'])

In [17]:
df.to_csv('../../Data/Results_solution_space/solutionSpace_bindingEnergy.tsv', 
         sep = '\t', index = False)

,new_binding_AB,new_binding_BB,cA,cB,cAA,cBB,cAB,pct_AB
0,-20.00,-20.00,0.000019,4.197775e-09,0.008213,0.008223,76.391796,99.978464
1,-20.00,-19.99,0.000019,4.215545e-09,0.008144,0.008154,76.391935,99.978644
2,-20.00,-19.98,0.000019,4.233390e-09,0.008076,0.008085,76.392072,99.978824
3,-20.00,-19.97,0.000019,4.251311e-09,0.008008,0.008017,76.392208,99.979002
4,-20.00,-19.96,0.000019,4.269307e-09,0.007940,0.007950,76.392342,99.979178
...,...,...,...,...,...,...,...,...
4409995,0.99,0.95,0.001330,1.259788e+01,38.200309,31.902029,0.006295,0.007611
4409996,0.99,0.96,0.001330,1.269503e+01,38.200284,31.853427,0.006344,0.007665
4409997,0.99,0.97,0.001330,1.279285e+01,38.200260,31.804503,0.006392,0.007720
4409998,0.99,0.98,0.001330,1.289133e+01,38.200235,31.755232,0.006442,0.007775


## Panels for figures 5 and 6

## Modify synthesis rates on the x-axis (log2 scale) and binding affinities on the y-axis

In [9]:
## Synthesis rate of A will be constant
sA = 100
log2_values = list(np.arange(-2, 2, 0.05))

sB_values = [sA * (2**float(entry)) for entry in log2_values]

## Binding affinity of AA and BB will be constant
curr_binding_energy_AA = -10
curr_binding_energy_BB = -10
binding_values_AB = np.arange(-13, -7, 0.1)


In [10]:
list_results = []
# Loop through synthesis rate values
# for new_sB in sB_values:
for i in range(len(sB_values)):
    new_sB = sB_values[i]
    log2_val = log2_values[i]

    syn_ratio = round(new_sB / sA, 3)
    
    for new_binding_AB in binding_values_AB:

        binding_diff = new_binding_AB - curr_binding_energy_AA
        
        # Feed the values into the system of equations
        cA, cB, cAA, cBB, cAB = equation_system(sA, new_sB, curr_stab_A, curr_stab_B, curr_binding_energy_AA, curr_binding_energy_BB, new_binding_AB)

        # Create a new line
        pct_AB = 100*cAB / (cA + cB + cAA + cAB + cBB)
        new_line = [sA, new_sB, curr_binding_energy_AA, curr_binding_energy_BB, new_binding_AB,
                    cA, cB, cAA, cBB, cAB, pct_AB, syn_ratio, binding_diff, log2_val]
        
        list_results.append(new_line)

    
# Convert the list of lists to a dataframe
df = pd.DataFrame(list_results, columns = ['sA', 'sB', 'binding_energy_AA', 'binding_energy_BB', 'binding_energy_AB',
                                           'cA', 'cB', 'cAA', 'cBB', 'cAB', 'pct_AB', 'syn_ratio', 'binding_diff', 'log2_val'])

In [11]:
df

,sA,sB,binding_energy_AA,binding_energy_BB,binding_energy_AB,cA,cB,cAA,cBB,cAB,pct_AB,syn_ratio,binding_diff,log2_val
0,100,25.000000,-10,-10,-13.0,0.001152,0.000002,28.652643,0.000127,19.101805,39.998982,0.250,-3.0,-2.00
1,100,25.000000,-10,-10,-12.9,0.001152,0.000003,28.652694,0.000177,19.101703,39.998768,0.250,-2.9,-2.00
2,100,25.000000,-10,-10,-12.8,0.001152,0.000003,28.652765,0.000249,19.101560,39.998468,0.250,-2.8,-2.00
3,100,25.000000,-10,-10,-12.7,0.001152,0.000004,28.652866,0.000349,19.101359,39.998048,0.250,-2.7,-2.00
4,100,25.000000,-10,-10,-12.6,0.001152,0.000005,28.653006,0.000489,19.101079,39.997459,0.250,-2.6,-2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,100,386.374532,-10,-10,-7.5,0.001311,0.002604,37.121658,146.527884,2.163616,1.164380,3.864,2.5,1.95
4796,100,386.374532,-10,-10,-7.4,0.001314,0.002606,37.287208,146.693435,1.832512,0.986191,3.864,2.6,1.95
4797,100,386.374532,-10,-10,-7.3,0.001316,0.002607,37.427758,146.833985,1.551410,0.834913,3.864,2.7,1.95
4798,100,386.374532,-10,-10,-7.2,0.001318,0.002608,37.546986,146.953214,1.312951,0.706583,3.864,2.8,1.95


In [30]:
## Save the dataframe
df.to_csv('../../Data/Results_solution_space/solutionSpace_syn_ratio_diff_binding_log2.tsv', 
         sep = '\t', index = False)

## Modify specific activities on the x-axis and binding affinities on the y-axis

In [13]:
## Synthesis rate of A will be constant
sA = 100
sB = 100

hetBias_values = np.arange(-80, 81, 1)

## Binding affinity of AA and BB will be constant
curr_binding_energy_AA = -10
curr_binding_energy_BB = -10
binding_values_AB = np.arange(-13, -7, 0.1)


In [20]:
list_results = []
# Loop through synthesis rate values
for hetBias in hetBias_values:

    # Use the hetBias value to calculate specific activities of HM and HET
    low_act_value = 1 - abs(hetBias / 100)
    if hetBias < 0:
        # The HET should have lower activity
        aAA = 1
        aAB = low_act_value
    elif hetBias > 0:
        # The HET should have higher activity
        aAA = low_act_value
        aAB = 1
    elif hetBias == 0:
        # Both have the same activity
        aAA = 1
        aAB = 1
    
    for new_binding_AB in binding_values_AB:

        binding_diff = new_binding_AB - curr_binding_energy_AA
        
        # Feed the values into the system of equations
        cA, cB, cAA, cBB, cAB = equation_system(sA, sB, curr_stab_A, curr_stab_B, curr_binding_energy_AA, curr_binding_energy_BB, new_binding_AB)

        # Create a new line
        pct_AB = 100*cAB / (cA + cB + cAA + cAB + cBB)
        total_activity = cAA*aAA + cBB*aAA + cAB*aAB
        
        new_line = [sA, sB, curr_binding_energy_AA, curr_binding_energy_BB, new_binding_AB,
                    cA, cB, cAA, cBB, cAB, pct_AB, binding_diff, hetBias, total_activity]
        
        list_results.append(new_line)
    
# Convert the list of lists to a dataframe
df = pd.DataFrame(list_results, columns = ['sA', 'sB', 'binding_energy_AA', 'binding_energy_BB', 'binding_energy_AB',
                                           'cA', 'cB', 'cAA', 'cBB', 'cAB', 'pct_AB', 'binding_diff', 
                                          'hetBias', 'total_activity'])

In [21]:
df

,sA,sB,binding_energy_AA,binding_energy_BB,binding_energy_AB,cA,cB,cAA,cBB,cAB,pct_AB,binding_diff,hetBias,total_activity
0,100,100,-10,-10,-13.0,0.000105,0.000105,0.239349,0.239349,75.929439,99.373224,-3.0,-80,15.664586
1,100,100,-10,-10,-12.9,0.000114,0.000114,0.283059,0.283059,75.842010,99.258790,-2.9,-80,15.734519
2,100,100,-10,-10,-12.8,0.000124,0.000124,0.334680,0.334680,75.738758,99.123644,-2.8,-80,15.817112
3,100,100,-10,-10,-12.7,0.000135,0.000135,0.395617,0.395617,75.616872,98.964112,-2.7,-80,15.914609
4,100,100,-10,-10,-12.6,0.000147,0.000147,0.467513,0.467513,75.473070,98.775894,-2.6,-80,16.029639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9655,100,100,-10,-10,-7.5,0.001320,0.001320,37.651186,37.651186,1.104549,1.445564,2.5,80,16.165024
9656,100,100,-10,-10,-7.4,0.001322,0.001322,37.735955,37.735955,0.935011,1.223683,2.6,80,16.029393
9657,100,100,-10,-10,-7.3,0.001323,0.001323,37.807849,37.807849,0.791222,1.035502,2.7,80,15.914362
9658,100,100,-10,-10,-7.2,0.001324,0.001324,37.868784,37.868784,0.669349,0.876002,2.8,80,15.816863


In [22]:
## Save the dataframe
df.to_csv('../../Data/Results_solution_space/solutionSpace_hetBias_diff_binding.tsv', 
         sep = '\t', index = False)